In [1]:
from importnb import Notebook

with Notebook():
    from DataClass import DataSet
    from DeepLabModel import Deeplabv3Plus
    from Utils import save_checkpoint, load_checkpoint, get_data_loaders,calculate_epoch_accuracy, calculate_validation_loss, calculate_miou
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2;
from tqdm import tqdm
from torch.optim.lr_scheduler import OneCycleLR

from torch.utils.tensorboard import SummaryWriter

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


###### Hyperparams

In [2]:
#LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#BATCH_SIZE = 8 #8
NUM_EPOCHS = 5
WEIGHT_DECAY = 1e-4
IMAGE_HEIGHT = 320 #320
IMAGE_WIDTH = 480 #480
LOAD_MODEL = False ##HEREE
TRAIN_IMG_DIR = 'C:\\Users\\georg\\Desktop\\Segmentation_data\\train\\images'
TRAIN_MASK_DIR = 'C:\\Users\\georg\\Desktop\\Segmentation_data\\train\\labels'
VAL_IMG_DIR = 'C:\\Users\\georg\\Desktop\\Segmentation_data\\validation\\images'
VAL_MASK_DIR = 'C:\\Users\\georg\\Desktop\\Segmentation_data\\validation\\lables'

In [3]:
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(), 
    ]
)

In [4]:
def train_fn(loader, model, optimizer, loss_fn, epoch, num_epochs):
    model.train()
    loop = tqdm(loader)
    

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.long().to(device=DEVICE)

        # forward pass
        predictions = model.forward(data)
        loss = loss_fn(predictions, targets)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #sched.step()
        
        # update tqdm loop
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
        
   

In [5]:
BATCH_SIZE = [4,8]
LEARNING_RATE = [1e-4,1e-3]

In [6]:

BATCH_SIZE = [4,8]
LEARNING_RATE = [1e-4,1e-3]
for BATCH_SIZE in BATCH_SIZE:
    for LEARNING_RATE in LEARNING_RATE:
        model = Deeplabv3Plus(10,False,False).to(DEVICE)
        step=0
        writer = SummaryWriter(f"runs/DeepLab/BS {BATCH_SIZE} LR {LEARNING_RATE}")
        loss_fn = nn.CrossEntropyLoss()
        train_loader, val_loader = get_data_loaders(
            TRAIN_IMG_DIR,
            TRAIN_MASK_DIR,
            VAL_IMG_DIR,
            VAL_MASK_DIR,
            BATCH_SIZE,
            train_transform,
            val_transform
        )
        optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        for epoch in range(NUM_EPOCHS):

            train_fn(train_loader, model, optimizer, loss_fn, epoch,NUM_EPOCHS)

            accuracy = calculate_epoch_accuracy(model, val_loader, device=DEVICE)
            loss = calculate_validation_loss(model, val_loader , loss_fn, device=DEVICE)
            miou = calculate_miou(model,val_loader,device=DEVICE,num_classes=10)

            writer.add_scalar("Loss/Validation",loss, global_step=step)

            writer.add_scalar("Acc/Validation",accuracy, global_step=step)
            writer.add_scalar("mIOU/Vallidation",miou, global_step=step)
            step+=1

            val_acc.append(accuracy)
            val_loss.append(loss)
            val_miou.append(miou)
            print(f"Accuracy {accuracy}, avg val loss {loss}, mIOU {miou} ")
        
        

C:\Users\georg\anaconda3\envs\Test_CV\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\georg\anaconda3\envs\Test_CV\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/225 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 261.25 MiB is free. Of the allocated memory 586.43 MiB is allocated by PyTorch, and 21.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)